In [51]:
import warnings

warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=FutureWarning)

In [52]:
import pandas as pd

supply_data = pd.read_csv("E:\mini project\supply_chain_data.csv")

In [53]:
supply_data.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Location,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs
0,haircare,SKU0,69.808006,55,802,8661.996792,Non-binary,58,7,96,...,Mumbai,29,215,29,46.279879,Pending,0.226410,Road,Route B,187.752075
1,skincare,SKU1,14.843523,95,736,7460.900065,Female,53,30,37,...,Mumbai,23,517,30,33.616769,Pending,4.854068,Road,Route B,503.065579
2,haircare,SKU2,11.319683,34,8,9577.749626,Unknown,1,10,88,...,Mumbai,12,971,27,30.688019,Pending,4.580593,Air,Route C,141.920282
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,Kolkata,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,Delhi,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632


In [54]:
supply_data.shape

(100, 24)

In [55]:
supply_data.columns

Index(['Product type', 'SKU', 'Price', 'Availability',
       'Number of products sold', 'Revenue generated', 'Customer demographics',
       'Stock levels', 'Lead times', 'Order quantities', 'Shipping times',
       'Shipping carriers', 'Shipping costs', 'Supplier name', 'Location',
       'Lead time', 'Production volumes', 'Manufacturing lead time',
       'Manufacturing costs', 'Inspection results', 'Defect rates',
       'Transportation modes', 'Routes', 'Costs'],
      dtype='object')

In [56]:
supply_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Product type             100 non-null    object 
 1   SKU                      100 non-null    object 
 2   Price                    100 non-null    float64
 3   Availability             100 non-null    int64  
 4   Number of products sold  100 non-null    int64  
 5   Revenue generated        100 non-null    float64
 6   Customer demographics    100 non-null    object 
 7   Stock levels             100 non-null    int64  
 8   Lead times               100 non-null    int64  
 9   Order quantities         100 non-null    int64  
 10  Shipping times           100 non-null    int64  
 11  Shipping carriers        100 non-null    object 
 12  Shipping costs           100 non-null    float64
 13  Supplier name            100 non-null    object 
 14  Location                 10

In [57]:
missing_values = supply_data.isnull().sum()


missing_values = missing_values[missing_values > 0]

if not missing_values.empty:
    print("Columns with missing values:")
    for column, count in missing_values.items():
        print(f"{column}: {count} missing values")
else:
    print("There are no columns with missing value")

There are no columns with missing value


In [58]:
if supply_data.duplicated().any():
    print(f"There are as many as {supply_data.duplicated().sum()} duplicate data.")
else:
    print("There are no duplicate data.")

There are no duplicate data.


In [59]:
import plotly.express as px

defect_rates_by_product = supply_data.groupby("Product type")['Defect rates'].mean().reset_index()

# Create a bar chart using Plotly
fig = px.bar(defect_rates_by_product, x='Product type', y='Defect rates', title='Defect Rates by Product Type')

# Customize the color scale for bars
color_scale = px.colors.qualitative.Set3  # You can choose a different color scale
fig.update_traces(marker_color=color_scale)

In [60]:
fig.update_layout(
    xaxis_title="Product Type",
    yaxis_title="Mean Defect Rates",
    xaxis=dict(categoryorder='total descending'),
    yaxis=dict(title='Mean Defect Rates'),
    plot_bgcolor='white',
    title_x=0.5,
    showlegend=True 
)


fig.show()

In [61]:
risk_data = supply_data[['SKU', 'Lead times', 'Stock levels']]
risk_data.head()

,SKU,Lead times,Stock levels
0,SKU0,7,58
1,SKU1,30,53
2,SKU2,10,1
3,SKU3,13,23
4,SKU4,3,5


In [62]:
risk_data['Risk score'] = risk_data['Lead times'] * (1-risk_data.loc[:,'Stock levels'])

In [63]:
risk_data = risk_data.sort_values(by='Risk score', ascending=False)[:10]


fig = px.bar(risk_data, x='SKU', y='Risk score', title='Top 10 Highest-Risk Data',
             labels={'Risk score': 'Risk Score', 'SKU': 'SKU'},
             text='Risk score')


fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')
fig.update_layout(xaxis_title='SKU', yaxis_title='Risk Score', title_x=0.5)


fig.show()


In [64]:
holdingcost = 0.2

In [65]:
import numpy as np

def calculate_eoq(data):
    S = data['Costs']
    D = data['Number of products sold'] 
    H = data['Number of products sold'] * holdingcost
    EOQ = np.sqrt((2*S*D)/H)
    return round(EOQ)

In [66]:
supply_data['EOQ'] = calculate_eoq(supply_data)

In [67]:
comparison_columns = supply_data[['SKU', 'EOQ','Order quantities']]
comparison_columns.head()

,SKU,EOQ,Order quantities
0,SKU0,43.0,96
1,SKU1,71.0,37
2,SKU2,38.0,88
3,SKU3,50.0,59
4,SKU4,96.0,56


In [68]:
top_10_comparison = comparison_columns.head(10)


fig = px.bar(top_10_comparison, x='SKU', y=['EOQ', 'Order quantities'], 
             title='Comparison of EOQ and Order Quantities for Top 10 SKUs')

fig.update_layout(
    title_x=0.5
)


fig.show()

In [69]:
revenue_avg_by_demo_prod = supply_data.groupby(['Customer demographics', 'Product type'])['Revenue generated'].mean().reset_index()
revenue_sum_by_demo_prod = supply_data.groupby(['Customer demographics', 'Product type'])['Revenue generated'].sum().reset_index()

In [70]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go


colors = px.colors.qualitative.Set3


fig = make_subplots(rows=1, cols=2, subplot_titles=('Average Revenue', 'Total Revenue'))


for i, product_type in enumerate(revenue_avg_by_demo_prod['Product type'].unique()):
    subset = revenue_avg_by_demo_prod[revenue_avg_by_demo_prod['Product type'] == product_type]
    fig.add_trace(
        go.Bar(
            x=subset['Customer demographics'],
            y=subset['Revenue generated'],
            name=product_type,
            marker_color=colors[i]
        ),
        row=1, col=1
    )


for i, product_type in enumerate(revenue_sum_by_demo_prod['Product type'].unique()):
    subset = revenue_sum_by_demo_prod[revenue_sum_by_demo_prod['Product type'] == product_type]
    fig.add_trace(
        go.Bar(
            x=subset['Customer demographics'],
            y=subset['Revenue generated'],
            showlegend=False,  
            marker_color=colors[i]
        ),
        row=1, col=2
    )


fig.update_layout(
    title='Revenue Analysis by Customer Demographics and Product Type',
    xaxis=dict(title='Customer Demographics'),
    yaxis=dict(title='Revenue'),
    xaxis2=dict(title='Customer Demographics'),
    yaxis2=dict(title='Revenue'),
    title_x=0.5,
    showlegend=True  
)


fig.show()

In [71]:
avg_lead_times_transport = supply_data.groupby(['Transportation modes'])['Lead times'].mean().reset_index()

fig = px.bar(avg_lead_times_transport, x='Transportation modes', y='Lead times',
             labels={'Transportation modes': 'Transportation Mode', 'Lead times': 'Average Lead Time'},
             title='Average Lead Times by Transportation Mode')


color_scale = px.colors.qualitative.Set3  
fig.update_traces(marker_color=color_scale)


fig.update_layout(
    title=dict(text='Average Lead Times by Transportation Mode', x=0.5),
    xaxis_title='Transportation Mode',
    yaxis_title='Average Lead Time',
    xaxis_tickangle=-45,  
    font=dict(family="Arial", size=14),
    showlegend=False  # Remove legend
)

fig.show()

In [72]:
best_transportation_mode = avg_lead_times_transport.loc[avg_lead_times_transport['Lead times'].idxmin()]
best_transportation_mode

Transportation modes          Sea
Lead times              12.176471
Name: 3, dtype: object

In [73]:
best_transportation_data = supply_data[supply_data['Transportation modes']==best_transportation_mode['Transportation modes']]
best_transportation_data.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs,EOQ
6,skincare,SKU6,4.078333,48,65,7823.476560,Male,11,15,58,...,14,314,24,1.085069,Pending,1.000911,Sea,Route A,134.369097,37.0
8,cosmetics,SKU8,68.717597,78,150,7517.363211,Female,5,10,15,...,13,769,8,11.423027,Pending,2.709863,Sea,Route B,505.557134,71.0
15,skincare,SKU15,36.989245,94,469,5442.086785,Non-binary,9,8,69,...,14,580,7,97.121282,Pass,2.264406,Sea,Route B,127.861800,36.0
18,haircare,SKU18,36.443628,23,620,9364.673505,Unknown,10,10,46,...,18,374,17,27.107981,Pending,2.231939,Sea,Route A,593.480259,77.0
27,cosmetics,SKU27,92.557361,42,352,2686.457224,Unknown,47,9,62,...,25,291,4,10.528245,Fail,2.864668,Sea,Route B,762.459182,87.0


In [74]:
import plotly.express as px

avg_lead_times_route = supply_data.groupby(['Routes'])['Lead times'].mean().reset_index()


fig = px.bar(avg_lead_times_route, x='Routes', y='Lead times',
             labels={'Routes': 'Route', 'Lead times': 'Average Lead Time'},
             title='Average Lead Times by Route')


color_scale = px.colors.qualitative.Set3  
fig.update_traces(marker_color=color_scale)


fig.update_layout(
    title=dict(text='Average Lead Times by Route', x=0.5),
    xaxis_title='Route',
    yaxis_title='Average Lead Time',
    xaxis_tickangle=-45,  
    font=dict(family="Arial", size=14),
    showlegend=False  
)
fig.show()


In [75]:
best_route = avg_lead_times_route.loc[avg_lead_times_route['Lead times'].idxmin()]
best_route

Routes          Route A
Lead times    14.697674
Name: 0, dtype: object

In [76]:
best_route_data = supply_data[supply_data['Routes']==best_route['Routes']]
best_route_data.head()

,Product type,SKU,Price,Availability,Number of products sold,Revenue generated,Customer demographics,Stock levels,Lead times,Order quantities,...,Lead time,Production volumes,Manufacturing lead time,Manufacturing costs,Inspection results,Defect rates,Transportation modes,Routes,Costs,EOQ
3,skincare,SKU3,61.163343,68,83,7766.836426,Non-binary,23,13,59,...,24,937,18,35.624741,Fail,4.746649,Rail,Route A,254.776159,50.0
4,skincare,SKU4,4.805496,26,871,2686.505152,Non-binary,5,3,56,...,5,414,3,92.065161,Fail,3.145580,Air,Route A,923.440632,96.0
5,haircare,SKU5,1.699976,87,147,2828.348746,Non-binary,90,27,66,...,10,104,17,56.766476,Fail,2.779194,Road,Route A,235.461237,49.0
6,skincare,SKU6,4.078333,48,65,7823.476560,Male,11,15,58,...,14,314,24,1.085069,Pending,1.000911,Sea,Route A,134.369097,37.0
11,skincare,SKU11,90.635460,95,960,6099.944116,Female,46,23,60,...,28,362,11,27.592363,Pending,0.021170,Air,Route A,126.723033,36.0


In [77]:
X = supply_data.loc[:,['Price', 'Availability', 'Stock levels', 'Lead times', 'Order quantities']]
y = supply_data.loc[:,'Number of products sold']

In [78]:
num_folds = 10

In [79]:
mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

In [80]:
from sklearn.model_selection import KFold

# Create a KFold object
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [81]:
params = {
    'objective': 'regression',
    'metric': 'mean_squared_error',
    'boosting_type': 'gbdt',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9
}

In [82]:
import lightgbm as lgb



In [83]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [84]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    
    train_data = lgb.Dataset(X_train, label=y_train)
    test_data = lgb.Dataset(X_test, label=y_test, reference=train_data)
    
    
    num_round = 100  
    

    
    bst = lgb.train(params,train_data,num_round,valid_sets=[test_data])
    
    
    y_pred = bst.predict(X_test, num_iteration=bst.best_iteration)

    
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    
    target_range = np.max(y_test) - np.min(y_test)

    
    percentage_mse = (mse / target_range) * 100
    percentage_rmse = (rmse / target_range) * 100
    percentage_mae = (mae / target_range) * 100
    percentage_r2 = (r2 * 100)

    
    mse_scores.append(percentage_mse)
    rmse_scores.append(percentage_rmse)
    mae_scores.append(percentage_mae)
    r2_scores.append(percentage_r2)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000084 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 135
[LightGBM] [Info] Number of data points in the train set: 90, number of used features: 5
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 449.877778
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

In [85]:
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)

print(f"Average Mean Squared Error: {avg_mse:.2f}%")
print(f"Average Root Mean Squared Error: {avg_rmse:.2f}%")
print(f"Average Mean Absolute Error: {avg_mae:.2f}%")
print(f"Average R-squared: {avg_r2:.2f}%")

Average Mean Squared Error: 12074.33%
Average Root Mean Squared Error: 37.43%
Average Mean Absolute Error: 32.81%
Average R-squared: -22.92%


In [86]:
X = supply_data.loc[:, 'Production volumes'].values.reshape(-1, 1)
y = supply_data.loc[:, 'Manufacturing costs'].values

In [87]:
num_folds = 5

In [88]:
mse_scores = []
rmse_scores = []
mae_scores = []
r2_scores = []

In [89]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(X)

MinMaxScaler()

In [90]:
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

In [91]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [92]:
for train_index, test_index in kf.split(X):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_dim=1),
        Dense(32, activation='relu'),
        Dense(1)
    ])

    model.compile(optimizer='adam', loss='mean_squared_error')

In [93]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    
model_checkpoint = ModelCheckpoint('/kaggle/working/best_model.h5', save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min')

    
history = model.fit(X_train_scaled,
                        y_train, epochs=100,
                        batch_size=32,
                        validation_data=(X_test_scaled, y_test),
                        callbacks=[early_stopping, model_checkpoint])

    # Load the best model weights from the saved checkpoint
model.load_weights('/kaggle/working/best_model.h5')
    
    # Make predictions on the test set
y_pred = model.predict(X_test_scaled)
    
    # Calculate metrics
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = np.mean(np.abs(y_test - y_pred))
r2 = r2_score(y_test, y_pred)

    # Calculate metrics as percentages
target_range = np.max(y_test) - np.min(y_test)
percentage_mse = (mse / target_range) * 100
percentage_rmse = (rmse / target_range) * 100
percentage_mae = (mae / target_range) * 100
percentage_r2 = (r2 * 100)

    # Append the scores to the respective lists
mse_scores.append(percentage_mse)
rmse_scores.append(percentage_rmse)
mae_scores.append(percentage_mae)
r2_scores.append(percentage_r2)


Epoch 1/100
3/3 [==============================] - 1s 123ms/step - loss: 3182.3091 - val_loss: 2563.9775
Epoch 2/100
3/3 [==============================] - 0s 33ms/step - loss: 3178.9868 - val_loss: 2560.7292
Epoch 3/100
3/3 [==============================] - 0s 35ms/step - loss: 3175.5962 - val_loss: 2557.3931
Epoch 4/100
3/3 [==============================] - 0s 31ms/step - loss: 3172.0671 - val_loss: 2553.9885
Epoch 5/100
3/3 [==============================] - 0s 30ms/step - loss: 3168.5898 - val_loss: 2550.3911
Epoch 6/100
3/3 [==============================] - 0s 30ms/step - loss: 3164.8257 - val_loss: 2546.5989
Epoch 7/100
3/3 [==============================] - 0s 39ms/step - loss: 3160.6401 - val_loss: 2542.5762
Epoch 8/100
3/3 [==============================] - 0s 31ms/step - loss: 3156.6055 - val_loss: 2538.3003
Epoch 9/100
3/3 [==============================] - 0s 34ms/step - loss: 3152.0662 - val_loss: 2533.8247
Epoch 10/100
3/3 [==============================] - 0s 32ms/ste

In [94]:
avg_mse = np.mean(mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_mae = np.mean(mae_scores)
avg_r2 = np.mean(r2_scores)

# Print the results
print(f"Average Mean Squared Error: {avg_mse:.2f}%")
print(f"Average Root Mean Squared Error: {avg_rmse:.2f}%")
print(f"Average Mean Absolute Error: {avg_mae:.2f}%")
print(f"Average R-squared: {avg_r2:.2f}%")

Average Mean Squared Error: 849.63%
Average Root Mean Squared Error: 30.04%
Average Mean Absolute Error: 25.03%
Average R-squared: -7.67%


In [95]:
min_production_volume = supply_data['Production volumes'].min()
max_production_volume = 1000
step_size = 100

cheapest_cost = float('inf')
best_production_volume = None

for production_volume in range(min_production_volume, max_production_volume + 1, step_size):
    normalized_production_volume = scaler.transform(np.array([[production_volume]]))
    predicted_cost = model.predict(normalized_production_volume)

    if predicted_cost[0][0] < cheapest_cost:
        cheapest_cost = predicted_cost[0][0]
        best_production_volume = production_volume

print('Most optimal production volume to minimize manufacturing cost:', best_production_volume)
print('The cheapest manufacturing cost:', cheapest_cost)

1/1 [==============================] - 0s 29ms/step
Most optimal production volume to minimize manufacturing cost: 104
The cheapest manufacturing cost: 23.00165
